# Working with Time Series Data

## About the Data
In this notebook, we will be working with 5 datasets:
- (CSV) Facebook's stock price daily throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- (CSV) Facebook's OHLC stock data from May 20, 2019 - May 24, 2019 per minute from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (CSV) melted stock data for Facebook from May 20, 2019 - May 24, 2019 per minute from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (DB) stock opening prices by the minute for Apple from May 20, 2019 - May 24, 2019 altered to have seconds in the time from [Nasdaq.com](https://old.nasdaq.com/symbol/aapl/interactive-chart).
- (DB) stock opening prices by the minute for Facebook from May 20, 2019 - May 24, 2019 from [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).

## Setup

In [1]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


## Time-based selection and filtering
Remember, when we have an index of type `DatetimeIndex`, we can use datetime slicing. We can provide a range of dates. We only get three days back because the stock market is closed on the weekends:

In [2]:
fb['2018-10-11':'2018-10-15']

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-10-11,150.13,154.81,149.1600,153.35,35338901,low
2018-10-12,156.73,156.89,151.2998,153.74,25293492,low
2018-10-15,153.32,155.57,152.5500,153.52,15433521,low


We can select ranges of months and quarters:

In [4]:
fb.loc['2018-q1'].equals(fb['2018-01':'2018-03'])

True

The `first()` method will give us a specified length of time from the beginning of the time series. Here, we ask for a week. January 1, 2018 was a holiday—meaning the market was closed. It was also a Monday, so the week here is only four days:

In [5]:
fb.first('1W')

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low


The `last()` method will take from the end:

In [6]:
fb.last('1W')

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-12-31,134.45,134.64,129.95,131.09,24625308,low


Suppose that we reindexed the Facebook stock data to include all dates for 2018. We would have null entries for January 1st:

In [7]:
fb_reindexed = fb.reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))
fb_reindexed.first('1D').isna().squeeze().all()

True

We can use `first_valid_index()` to give us the index of the first non-null entry in our data, which is the first day the market was open in Q1 2018:

In [9]:
fb_reindexed.loc['2018-Q1'].first_valid_index()

Timestamp('2018-01-02 00:00:00', freq='D')

Conversely, we can use `last_valid_index()` to get the last entry of non-null data. For Q1 2018, this is March 29th:

In [10]:
fb_reindexed.loc['2018-Q1'].last_valid_index()

Timestamp('2018-03-29 00:00:00', freq='D')

We can use `asof()` to find the last non-null data before the point we are looking for. If we ask for March 31st, we will get the data from the index we got from `fb_reindexed.loc['2018-Q1'].last_valid_index()`, which was March 29th. Note that this works regardless of whether we reindexed:

In [11]:
fb_reindexed.asof('2018-03-31')

open                   155.15
high                   161.42
low                    154.14
close                  159.79
volume            59434293.00
trading_volume            low
Name: 2018-03-31 00:00:00, dtype: object

For the next few examples, we need datetimes, so we will read in the stock data per minute file:

In [13]:
stock_data_per_minute = pd.read_csv(
    'data/fb_week_of_may_20_per_minute.csv', index_col='date', parse_dates=True, 
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)

stock_data_per_minute.head()

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.6200,181.6200,181.6200,181.6200,159049.0
2019-05-20 09:31:00,182.6100,182.6100,182.6100,182.6100,468017.0
2019-05-20 09:32:00,182.7458,182.7458,182.7458,182.7458,97258.0
2019-05-20 09:33:00,182.9500,182.9500,182.9500,182.9500,43961.0
2019-05-20 09:34:00,183.0600,183.0600,183.0600,183.0600,79562.0


We can use a `Grouper` object to roll up our data to the daily level along with `first` and `last`:

In [14]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

,open,high,low,close,volume
date,,,,,
2019-05-20,181.62,184.1800,181.6200,182.72,10044838.0
2019-05-21,184.53,185.5800,183.9700,184.82,7198405.0
2019-05-22,184.81,186.5603,184.0120,185.32,8412433.0
2019-05-23,182.50,183.7300,179.7559,180.87,12479171.0
2019-05-24,182.33,183.5227,181.0400,181.06,7686030.0


The `at_time()` method allows us to pull out all datetimes that match a certain time. Here, we can grab all the rows from the time the stock market opens (9:30 AM):

In [15]:
stock_data_per_minute.at_time('9:30')

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.62,181.62,181.62,181.62,159049.0
2019-05-21 09:30:00,184.53,184.53,184.53,184.53,58171.0
2019-05-22 09:30:00,184.81,184.81,184.81,184.81,41585.0
2019-05-23 09:30:00,182.50,182.50,182.50,182.50,121930.0
2019-05-24 09:30:00,182.33,182.33,182.33,182.33,52681.0


We can use `between_time()` to grab data for the last two minutes of trading daily:

In [16]:
stock_data_per_minute.between_time('15:59', '16:00')

,open,high,low,close,volume
date,,,,,
2019-05-20 15:59:00,182.915,182.915,182.915,182.915,134569.0
2019-05-20 16:00:00,182.720,182.720,182.720,182.720,1113672.0
2019-05-21 15:59:00,184.840,184.840,184.840,184.840,61606.0
2019-05-21 16:00:00,184.820,184.820,184.820,184.820,801080.0
2019-05-22 15:59:00,185.290,185.290,185.290,185.290,96099.0
2019-05-22 16:00:00,185.320,185.320,185.320,185.320,1220993.0
2019-05-23 15:59:00,180.720,180.720,180.720,180.720,109648.0
2019-05-23 16:00:00,180.870,180.870,180.870,180.870,1329217.0
2019-05-24 15:59:00,181.070,181.070,181.070,181.070,52994.0


On average, are more shares traded within the first 30 minutes of trading or in the last 30 minutes? We can combine `between_time()` with `group_by()` and `filter()` from the [`3-aggregations.ipynb`](./3-aggregations.ipynb) notebook to answer this question. For the week in question, more are traded on average around opening time than closing time:

In [17]:
shares_traded_in_first_30_min = stock_data_per_minute\
    .between_time('9:30', '10:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_last_30_min = stock_data_per_minute\
    .between_time('15:30', '16:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_first_30_min - shares_traded_in_last_30_min

18592.967741935485

In cases where time doesn't matter, we can normalize the times to midnight:

In [ ]:
pd.DataFrame(
    dict(before=stock_data_per_minute.index, after=stock_data_per_minute.index.normalize())
).head()

Note that we can also use `normalize()` on a `Series` object after accessing the `dt` attribute:

In [ ]:
stock_data_per_minute.index.to_series().dt.normalize().head()

## Shifting for lagged data
We can use `shift()` to create lagged data. By default, the shift will be one period. For example, we can use `shift()` to create a new column that indicates the previous day's closing price. From this new column, we can calculate the price change due to after-hours trading (after the close one day right up to the open the following day):

In [ ]:
fb.assign(
    prior_close=lambda x: x.close.shift(),
    after_hours_change_in_price=lambda x: x.open - x.prior_close,
    abs_change=lambda x: x.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

If the goal is to to add/subtract time, we can use `pd.Timedelta` objects instead:

In [ ]:
pd.date_range('2018-01-01', freq='D', periods=5) + pd.Timedelta('9 hours 30 minutes')

## Differenced data
Using the `diff()` method is a quick way to calculate the difference between the data and a lagged version of itself. By default, it will yield the result of `data - data.shift()`:

In [ ]:
(
    fb.drop(columns='trading_volume') 
    - fb.drop(columns='trading_volume').shift()
).equals(
    fb.drop(columns='trading_volume').diff()
)

We can use this to see how Facebook stock changed day-over-day:

In [ ]:
fb.drop(columns='trading_volume').diff().head()

We can specify the number of periods, can be any positive or negative integer:

In [ ]:
fb.drop(columns='trading_volume').diff(-3).head()

## Resampling
Sometimes the data is at a granularity that isn't conducive to our analysis. Consider the case where we have data per minute for the full year of 2018. Let's see what happens if we try to plot this, and then look at the daily aggregation of this data.

*Plotting will be covered next chapter.*

In [ ]:
from visual_aids.misc_viz import resampling_example
resampling_example()

The plot on the left has so much data we can't see anything. However, when we aggregate to the daily totals, we see the data. We can alter the granularity of the data we are working with using resampling. Recall our minute-by-minute stock data:

In [ ]:
stock_data_per_minute.head()

We can resample this to get to a daily frequency:

In [ ]:
stock_data_per_minute.resample('1D').agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

We can downsample to quarterly data:

In [ ]:
fb.resample('Q').mean()

We can also use `apply()`. Here, we show the quarterly change from start to end:

In [ ]:
fb.drop(columns='trading_volume').resample('Q').apply(
    lambda x: x.last('1D').values - x.first('1D').values
)

Consider the following melted stock data by the minute. We don't see the OHLC data directly:

In [ ]:
melted_stock_data = pd.read_csv('data/melted_stock_data.csv', index_col='date', parse_dates=True)
melted_stock_data.head()

We can use the `ohlc()` method after resampling to recover the OHLC columns:

In [ ]:
melted_stock_data.resample('1D').ohlc()['price']

Alternatively, we can upsample to increase the granularity. Note this will introduce `NaN` values:

In [ ]:
fb.resample('6H').asfreq().head()

There are many ways to handle these `NaN` values. We can forward-fill with `pad()`:

In [ ]:
fb.resample('6H').pad().head()

We can specify a specific value or a method with `fillna()`:

In [ ]:
fb.resample('6H').fillna('nearest').head()

We can use `asfreq()` and `assign()` to specify the action per column:

In [ ]:
fb.resample('6H').asfreq().assign(
    volume=lambda x: x.volume.fillna(0), # put 0 when market is closed
    close=lambda x: x.close.fillna(method='ffill'), # carry forward
    # take the closing price if these aren't available
    open=lambda x: x.open.combine_first(x.close),
    high=lambda x: x.high.combine_first(x.close),
    low=lambda x: x.low.combine_first(x.close)
).head()

## Merging
We saw merging examples the [`1-querying_and_merging.ipynb`](./1-querying_and_merging.ipynb) notebook. However, they all matched based on keys. With time series, it is possible that they are so granular that we never have the same time for multiple entries. Let's work with some stock data at different granularities:

In [ ]:
import sqlite3

with sqlite3.connect('data/stocks.db') as connection:
    fb_prices = pd.read_sql(
        'SELECT * FROM fb_prices', connection, 
        index_col='date', parse_dates=['date']
    )
    aapl_prices = pd.read_sql(
        'SELECT * FROM aapl_prices', connection, 
        index_col='date', parse_dates=['date']
    )

The Facebook prices are at the minute granularity:

In [ ]:
fb_prices.index.second.unique()

However, the Apple prices have information for the second:

In [ ]:
aapl_prices.index.second.unique()

We can perform an *as of* merge to try to line these up the best we can. We specify how to handle the mismatch with the `direction` and `tolerance` parameters. We will fill in with the `direction` of `nearest` and a `tolerance` of 30 seconds. This will place the Apple data with the minute that it is closest to, so 9:31:52 will go with 9:32 and 9:37:07 will go with 9:37. Since the times are on the index, we pass in `left_index` and `right_index`, as we did with `merge()` earlier this chapter: 

In [ ]:
pd.merge_asof(
    fb_prices, aapl_prices, 
    left_index=True, right_index=True, # datetimes are in the index
    # merge with nearest minute
    direction='nearest', tolerance=pd.Timedelta(30, unit='s')
).head()

If we don't want to lose the seconds information with the Apple data, we can use `pd.merge_ordered()` instead, which will interleave the two. Note this is an outer join by default (`how` parameter). The only catch here is that we need to reset the index in order to join on it:

In [ ]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

We can pass a `fill_method` to handle `NaN` values:

In [ ]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index(),
    fill_method='ffill'
).set_index('date').head()

Alternatively, we can use `fillna()`.

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="./3-aggregations.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_04/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_05/1-introducing_matplotlib.ipynb">
            <button>Chapter 5 &#8594;</button>
        </a>
    </div>
</div>
<hr>